### Imports

In [1]:
import pandas as pd

In [2]:
# pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Leemos y consolidamos los datasets

In [3]:
ventas = pd.read_csv('../Datasets/sell-in.txt', sep='\t')

productos = pd.read_csv('../Datasets/tb_productos.txt', sep='\t')
productos = productos.drop_duplicates(subset=['product_id'])

stock = pd.read_csv('../Datasets/tb_stocks.txt', sep='\t')
productos_predecir = pd.read_csv('../Datasets/productos_a_predecir.csv')

ventas_grouped = ventas.groupby(['product_id', 'periodo']).agg({
    'plan_precios_cuidados': 'sum',
    'cust_request_qty': 'sum', # sirve de algo para el modelo? En zulip se me menciona que es la cantidad de solicitudes que hace el cliente, por lo que creo que no
    'cust_request_tn': 'sum',
    'tn': 'sum'
}).reset_index()

ventas_producto = pd.merge(ventas_grouped, productos, on=['product_id'], how='inner')

final_dataset = pd.merge(ventas_producto, stock, on=['product_id', 'periodo'], how='left')
final_dataset = pd.merge(final_dataset, productos_predecir, on=['product_id'], how='inner')

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

In [4]:
duplicates = final_dataset.duplicated(subset=['product_id', 'periodo'])
duplicate_rows = final_dataset[duplicates]

if duplicate_rows.empty:
    print("No hay registros duplicados por product_id y periodo.")
else:
    print("Registros duplicados encontrados por product_id y periodo:")
    display(duplicate_rows)


No hay registros duplicados por product_id y periodo.


In [5]:
final_dataset.isnull().sum()

product_id                   0
periodo                      0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
tn                           0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
descripcion                  0
stock_final              11622
dtype: int64

### Analisis exploratorio basico

#### Analisis inicial

In [6]:
final_dataset.shape

(22349, 13)

In [7]:
final_dataset.head(20)

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final
0,20001,201701,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
1,20001,201702,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
2,20001,201703,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
3,20001,201704,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
4,20001,201705,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
5,20001,201706,0,570,1575.82891,1520.06539,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
6,20001,201707,0,381,1086.47101,1030.67391,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
7,20001,201708,0,643,1289.66869,1267.39462,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
8,20001,201709,0,381,1356.96103,1316.94604,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
9,20001,201710,0,273,1441.60247,1439.75563,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN


In [8]:
final_dataset[final_dataset['cust_request_tn'] != final_dataset['tn']]

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final
0,20001,201701,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
1,20001,201702,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
2,20001,201703,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
3,20001,201704,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
4,20001,201705,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22062,21218,201907,0,17,0.09282,0.08153,REF,TE,Hierbas,TWININGS,20,Menta,0.61953
22097,21222,201906,0,15,0.04368,0.03931,REF,TE,Verde,TWININGS,20,Verde con Menta,0.53945
22120,21224,201806,0,128,0.18116,0.18093,PC,DEOS,Aero,DEOS3,50,Sabor 5,NaN
22132,21224,201906,0,193,0.26413,0.26390,PC,DEOS,Aero,DEOS3,50,Sabor 5,0.22244


In [9]:
final_dataset.describe()

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,sku_size,stock_final
count,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,22349.000000,10727.000000
mean,20479.013423,201817.913687,1.249720,222.769744,51.364013,50.230669,447.639134,21.202147
std,334.475371,81.433326,12.981709,123.774437,129.425314,125.016406,831.235708,60.538364
min,20001.000000,201701.000000,0.000000,1.000000,0.000890,0.000890,1.000000,-13.666560
25%,20200.000000,201711.000000,0.000000,129.000000,3.010850,2.991610,89.000000,1.422785
50%,20411.000000,201809.000000,0.000000,207.000000,11.936620,11.900460,220.000000,6.120440
75%,20730.000000,201905.000000,0.000000,304.000000,36.272670,35.983630,475.000000,19.136220
max,21276.000000,201912.000000,289.000000,756.000000,2423.708810,2295.198320,10000.000000,1562.024480


In [10]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final
396,20012,201701,172,386,491.10893,476.39728,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
397,20012,201702,185,349,554.77790,509.04048,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
398,20012,201703,205,567,665.79620,649.49079,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
399,20012,201704,190,496,755.65972,749.91517,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
932,20026,201909,143,359,210.99220,209.49245,HC,ROPA LAVADO,Liquido,LIMPIEX,800,Maquina 1er lavado,27.12045
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21482,21155,201902,82,135,0.35821,0.35821,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.13209
21483,21155,201903,90,166,0.52102,0.52102,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.46664
21484,21155,201904,70,126,0.51206,0.51206,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.16727
21485,21155,201905,65,103,0.48436,0.48436,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.27902


#### Analizo si puede existir alguna venta de un producto en el mismo periodo que no sea plan_precios_cuidados y otras que si

In [11]:
ventas[(ventas['product_id'] == 20026) & (ventas['periodo'] == 201909) & (ventas['plan_precios_cuidados'] == 0)]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


In [12]:
def has_both_types(group):
    has_cuidados = (group['plan_precios_cuidados'] != 0).any()
    has_no_cuidados = (group['plan_precios_cuidados'] == 0).any()
    # display(group)
    return has_cuidados and has_no_cuidados

filtered_groups = ventas.groupby(['product_id', 'periodo']).filter(has_both_types)
filtered_groups

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn


Como observo que no existe ningun producto, en ningun periodo, que fuera a su vez precio_cuidoda y que no lo fuera, procedo a modificar esa variable en dataset_final para transformarla en boolean en vez de en suma:

In [13]:
final_dataset['plan_precios_cuidados'] = final_dataset['plan_precios_cuidados'].apply(lambda x: 1 if x > 0 else 0)

In [14]:
final_dataset[(final_dataset['plan_precios_cuidados'] != 0) & (~final_dataset['plan_precios_cuidados'].isna())]

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final
396,20012,201701,1,386,491.10893,476.39728,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
397,20012,201702,1,349,554.77790,509.04048,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
398,20012,201703,1,567,665.79620,649.49079,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
399,20012,201704,1,496,755.65972,749.91517,HC,ROPA ACONDICIONADOR,ACONDICIONADOR,DOWNY,800,Sabor 15,NaN
932,20026,201909,1,359,210.99220,209.49245,HC,ROPA LAVADO,Liquido,LIMPIEX,800,Maquina 1er lavado,27.12045
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21482,21155,201902,1,135,0.35821,0.35821,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.13209
21483,21155,201903,1,166,0.52102,0.52102,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.46664
21484,21155,201904,1,126,0.51206,0.51206,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.16727
21485,21155,201905,1,103,0.48436,0.48436,HC,HOGAR,Esponja,MUSCULO,1,Salva UÃ±as,0.27902


In [15]:
# Observo que tambien los Na de plan_precios_cuidados se mapearon a 0
final_dataset[final_dataset['plan_precios_cuidados'].isna()]

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final


#### Defino la variable target

In [16]:
final_dataset.rename(columns={'tn': 'y'}, inplace=True)
final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)
final_dataset

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final
0,20001,201701,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
1,20001,201702,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
2,20001,201703,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
3,20001,201704,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
4,20001,201705,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22344,21276,201908,0,7,0.01265,0.01265,PC,PIEL1,Cara,NIVEA,140,reconstruccion,1.08488
22345,21276,201909,0,10,0.01856,0.01856,PC,PIEL1,Cara,NIVEA,140,reconstruccion,0.87622
22346,21276,201910,0,13,0.02079,0.02079,PC,PIEL1,Cara,NIVEA,140,reconstruccion,1.05889
22347,21276,201911,0,12,0.03341,0.03341,PC,PIEL1,Cara,NIVEA,140,reconstruccion,1.06112


#### Analisis nulos

In [17]:
# TODO
final_dataset.isnull().sum()

product_id                   0
periodo                      0
plan_precios_cuidados        0
cust_request_qty             0
cust_request_tn              0
y                            0
cat1                         0
cat2                         0
cat3                         0
brand                        0
sku_size                     0
descripcion                  0
stock_final              11622
dtype: int64

El stock no parece ser util para los modelos, y es a su vez, la unica variable con valores nulos, por lo que no haremos ninguna tratamiento de momento.

#### Feature Engineering

Agregamos una columna que indica si el mes es cierre de Q, y otra columna que indique la "edad" del producto, considerando la fecha de la primer venta en el dataset

In [18]:
final_dataset['periodo_temp'] = final_dataset['periodo'].astype(str)
final_dataset['close_quarter'] = final_dataset['periodo_temp'].str[-2:].isin(['03', '06', '09', '12']).astype(int)

final_dataset['first_period'] = final_dataset.groupby('product_id')['periodo'].transform('min')
final_dataset['first_period'] = final_dataset['first_period'].astype(str)

final_dataset['age'] = ((final_dataset['periodo_temp'].str[:4].astype(int) - final_dataset['first_period'].str[:4].astype(int)) * 12 +
                        (final_dataset['periodo_temp'].str[-2:].astype(int) - final_dataset['first_period'].str[-2:].astype(int)))


final_dataset.drop(columns=['periodo_temp', 'first_period'], inplace=True)
display(final_dataset.head())

final_dataset.to_csv('../Datasets/final_dataset.csv', sep='\t', index=False)

,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,y,cat1,cat2,cat3,brand,sku_size,descripcion,stock_final,close_quarter,age
0,20001,201701,0,479,937.72717,934.77222,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN,0,0
1,20001,201702,0,432,833.72187,798.01620,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN,0,1
2,20001,201703,0,509,1330.74697,1303.35771,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN,1,2
3,20001,201704,0,279,1132.94430,1069.96130,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN,0,3
4,20001,201705,0,701,1550.68936,1502.20132,HC,ROPA LAVADO,Liquido,ARIEL,3000,genoma,NaN,0,4


#### Generacion de file para kaggle (para usar en los modelos)

In [19]:
#generar dataset Kaggle
#df.to_csv('../Datasets/df.csv', sep=',', index=False)

#### Formula para calcular el error (para usar en los modelos)

In [20]:
# df_mm['AbsoluteError'] = abs(df_mm['y'] - df_mm['prediction'])
# total_forecast_error = df_mm['AbsoluteError'].sum() / df_mm['y'].sum()